In [ ]:
!python3 -m scripts.gesture_recognition_cnn.train_dynamic

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Epoch [1/1000] Train Loss: 1.3579 Train Acc: 46.29% Val Loss: 0.8423 Val Acc: 67.21%
Best model saved with Val Acc: 67.21%
Epoch [2/1000] Train Loss: 0.7751 Train Acc: 68.80% Val Loss: 0.5361 Val Acc: 81.15%
Best model saved with Val Acc: 81.15%
Epoch [3/1000] Train Loss: 0.5393 Train Acc: 79.22% Val Loss: 0.4245 Val Acc: 86.48%
Best model saved with Val Acc: 86.48%
Epoch [4

In [ ]:
!python3 -m scripts.hand_tracking.train_yolo